In [23]:
import requests
import pandas as pd
from datetime import datetime as dt

base_url = "https://api.openweathermap.org/data/2.5/forecast?"
API_KEY = "2c31da6690d1c4df5a807fd44ab9fe80"
city = "London"

url = base_url + "appid=" + API_KEY + "&q=" + city

response = requests.get(url).json()

print(response)



{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1755529200, 'main': {'temp': 294.34, 'feels_like': 294.16, 'temp_min': 294.34, 'temp_max': 295.38, 'pressure': 1019, 'sea_level': 1019, 'grnd_level': 1015, 'humidity': 63, 'temp_kf': -1.04}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.06, 'deg': 87, 'gust': 4.9}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-08-18 15:00:00'}, {'dt': 1755540000, 'main': {'temp': 294.22, 'feels_like': 293.95, 'temp_min': 293.97, 'temp_max': 294.22, 'pressure': 1019, 'sea_level': 1019, 'grnd_level': 1014, 'humidity': 60, 'temp_kf': 0.25}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'clouds': {'all': 78}, 'wind': {'speed': 4.35, 'deg': 87, 'gust': 5.56}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-08-18 18:00:00'}, {'dt': 1755550800, 'main': {'temp': 292.45, 'feels_lik

In [8]:
import requests                                 #used to send and receive http requests
import pandas as pd                          #used to manipulate data and create dataframes
from datetime import datetime, timezone     #used to work with dates and times

API_KEY = "2c31da6690d1c4df5a807fd44ab9fe80"   # my key (should be kept private)
UNITS = "metric"                               # Celsius
CITIES = ["Miami", "Jakarta", "Singapore"]

def geocode(city: str):         #Get lat/lon for a city as city names can be for multiple locations
    r = requests.get(       #sends a reuqest to the geocoding API to get the latitude and longitude of the city
        "https://api.openweathermap.org/geo/1.0/direct",
        params={"q": city, "limit": 1, "appid": API_KEY},       #API call of city and limit 1 selects the largest match
    )
    js = r.json()                                       #converts the response to a JSON object
    return js[0]["lat"], js[0]["lon"]               #returns the latitude and longitude of the city

def daily_from_5day(lat, lon):              #uses coordinates to get the 5 day forecast
    r = requests.get(
        "https://api.openweathermap.org/data/2.5/forecast",
        params={"lat": lat, "lon": lon, "units": UNITS, "appid": API_KEY},
    )
    data = r.json()

    rows = []                                                   # list to hold daily temperature data
    for it in data["list"]:                                     # iterate through the 5-day forecast data                           
        ts = datetime.fromtimestamp(it["dt"], tz=timezone.utc)  #convert the timestamp to a datetime object
        rows.append({                                           # dictionary to hold the date and temperature data
            "date": ts.date(),
            "tmin": it["main"]["temp_min"],
            "tmax": it["main"]["temp_max"],
        })

    df = pd.DataFrame(rows)                        # create a DataFrame from the list of dictionaries       
    daily = (
        df.groupby("date")              # group by date to aggregate daily temperatures
        .agg(tmin=("tmin", "min"), tmax=("tmax", "max"))    # aggregate to daily min/max temperatures
        .reset_index()
        .sort_values("date")
    )
    return daily

def next_days_only(df: pd.DataFrame):           #Keep days strictly after today (tomorrow onward).
    today = datetime.now(timezone.utc).date()
    return df[df["date"] > today].reset_index(drop=True)

# run for each city
all_out = []
for city in CITIES:
    lat, lon = geocode(city)
    df = daily_from_5day(lat, lon)  #fetch daily forecast data
    df = next_days_only(df)         #keep only future dates
    df.insert(0, "city", city)  #insert the city name as the first column
    all_out.append(df)             #save each city's data




combined = pd.concat(all_out, ignore_index=True)
combined.to_csv("forecast.csv", index=False)
print(combined)


         city        date   tmin   tmax
0       Miami  2025-08-21  29.32  35.67
1       Miami  2025-08-22  29.28  33.96
2       Miami  2025-08-23  29.42  33.83
3       Miami  2025-08-24  25.77  32.03
4       Miami  2025-08-25  29.13  32.64
5     Jakarta  2025-08-21  25.44  28.77
6     Jakarta  2025-08-22  25.40  31.02
7     Jakarta  2025-08-23  25.55  30.42
8     Jakarta  2025-08-24  25.82  31.33
9     Jakarta  2025-08-25  26.06  31.23
10  Singapore  2025-08-21  25.72  26.84
11  Singapore  2025-08-22  26.84  27.43
12  Singapore  2025-08-23  26.95  27.75
13  Singapore  2025-08-24  27.25  27.60
14  Singapore  2025-08-25  27.53  28.42
